<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/All_in_One_Indicators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas-ta
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import yfinance as yfin
yfin.pdr_override()
import pandas as pd
import pandas_ta as ta

import numpy as np
import plotly.graph_objs as go
import plotly
import plotly.subplots

def get_all_in_one_indicators(stock):
  df = yfin.Ticker(stock).history(period='1y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  # Force lowercase (optional)
  df.columns = [x.lower() for x in df.columns]
  # calculate MACD values
  df['200_ema'] = round(df['close'].ewm(span = 200, adjust = False).mean(),0) 
  df['50_ema'] = round(df['close'].ewm(span = 50, adjust = False).mean(),0) 
  df.ta.macd(close='close', fast=12, slow=26, append=True)
  df.ta.eom(append=True)
  df.ta.rsi(append=True)

  df = df[df.index > '2022-03-31']
  # Force lowercase (optional)
  df.columns = [x.lower() for x in df.columns]
  # Construct a 2 x 1 Plotly figure
  fig = plotly.subplots.make_subplots(rows=4, cols=1)
  # price Line
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['close'],
          line=dict(color='#0000ff', width=1),
          name='price',
          # showlegend=False,
          # legendgroup='1',
      ), row=1, col=1
  )
  # EMA 200
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['200_ema'],
          line=dict(color='#000000', width=2),
          name='200 ema',
          # showlegend=False,
          # legendgroup='2',
      ), row=1, col=1
  )
  # EMA 50
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['50_ema'],
          line=dict(color='#ff9900', width=2),
          name='50 ema',
          # showlegend=False,
          # legendgroup='2',
      ), row=1, col=1
  )
  # Fast Signal (%k)
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['macd_12_26_9'],
          line=dict(color='#ff9900', width=2),
          name='macd',
          # showlegend=False,
          # legendgroup='2',
      ), row=2, col=1
  )
  # Slow signal (%d)
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['macds_12_26_9'],
          line=dict(color='#000000', width=2),
          # showlegend=False,
          # legendgroup='2',
          name='signal'
      ), row=2, col=1
  )
  # Colorize the histogram values
  colors = np.where(df['macdh_12_26_9'] < 0, '#000', '#ff9900')
  # Plot the histogram
  fig.append_trace(
      go.Bar(
          x=df.index,
          y=df['macdh_12_26_9'],
          name='histogram',
          marker_color=colors,
      ), row=2, col=1
  )
  # RSI
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['rsi_14'],
          line=dict(color='#00ff00', width=2),
          name='rsi',
          # showlegend=False,
          legendgroup='2',
      ), row=3, col=1
  )
  # EOM
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['eom_14_100000000'],
          line=dict(color='#ff0000', width=2),
          name='eom',
          # showlegend=False,
          legendgroup='2',
      ), row=4, col=1
  )
  # Make it pretty
  layout = go.Layout(
      plot_bgcolor='#efefef',
      # Font Families
      font_family='Monospace',
      font_color='#000000',
      font_size=12,
      width=1500, 
      height=1000,
      xaxis=dict(
          rangeslider=dict(
              visible=False
          )
      )
  )
  # Update options and show plot
  fig.update_layout(layout)
  fig.show()

stock = 'BAJFINANCE.NS'
get_all_in_one_indicators(stock)

In [16]:
df = yfin.Ticker(stock).history(period='1y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
# Force lowercase (optional)
df.columns = [x.lower() for x in df.columns]
# calculate MACD values
df.ta.macd(close='close', fast=12, slow=26, append=True)


,open,close,low,high,volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
Date,,,,,,,,
2021-12-27 00:00:00+05:30,6726.507977,6865.750977,6629.909508,6883.735298,912360,NaN,NaN,NaN
2021-12-28 00:00:00+05:30,6954.677252,6881.593262,6836.856252,6954.677252,862464,NaN,NaN,NaN
2021-12-29 00:00:00+05:30,6859.922241,6886.575195,6828.187692,6924.736539,532379,NaN,NaN,NaN
2021-12-30 00:00:00+05:30,6874.867685,6846.172363,6832.023685,6919.754539,658992,NaN,NaN,NaN
2021-12-31 00:00:00+05:30,6857.032750,6951.986816,6850.605956,6973.608285,769907,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-12-20 00:00:00+05:30,6660.000000,6642.850098,6585.000000,6698.850098,607716,-86.199764,20.127654,-106.327419
2022-12-21 00:00:00+05:30,6693.899902,6526.700195,6505.000000,6693.899902,622390,-89.271094,13.645060,-102.916154
2022-12-22 00:00:00+05:30,6570.000000,6490.649902,6407.000000,6575.000000,849706,-93.535879,7.504220,-101.040099
